### <span style="color:#FF00FF">Named-Entity Recognition (NER) using Keras Bidirectional LSTM</span>

### <span style="color:#FF00FF">Import libraries</span>

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

from keras.utils import to_categorical
from keras.models import load_model

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

### <span style="color:#FF00FF">Load the dataset</span>

In [2]:
df = pd.read_csv("../data/jobdescriptions_input.csv")
df.head()

,sentence_id,words,labels
0,0,manage,O
1,0,verticals,O
2,0,company,O
3,0,like,O
4,0,applications,I-SKILL


### <span style="color:#FF00FF">Number of unique words in the corpus and tags</span>

In [3]:
custom_labels = df["labels"].unique().tolist()
custom_labels

['O', 'I-SKILL', 'I-DEPT', 'I-ROLE', 'B-ROLE', 'B-SKILL', 'B-DEPT']

In [4]:
print("Unique words in corpus:", df['words'].nunique())
print("Unique tags in corpus:", df['labels'].nunique())

Unique words in corpus: 10165
Unique tags in corpus: 7


No. of tags in corpus dataset is directly proportional to the number of classes which is 7  in our case and our input dimension is 10165.

### <span style="color:#FF00FF">Retrieve Sentences and Corresponding Tags</span>

In [5]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["words"].values.tolist(),
                                                        s["labels"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [6]:
getter = SentenceGetter(df)
sentences = getter.sentences
print(sentences[0])

[('manage', 'O'), ('verticals', 'O'), ('company', 'O'), ('like', 'O'), ('applications', 'I-SKILL'), ('infrastructure', 'I-DEPT'), ('security', 'O'), ('leverage', 'O'), ('shared', 'O'), ('service', 'O'), ('organization', 'O'), ('deployment', 'I-SKILL'), ('new', 'O'), ('solutions', 'O'), ('develop', 'O'), ('organization', 'O'), ('level', 'O'), ('strategies', 'O'), ('scenario', 'O'), ('align', 'O'), ('business', 'O'), ('objectives', 'O'), ('cost', 'O'), ('efficient', 'O'), ('manner', 'O'), ('analyse', 'O'), ('existing', 'O'), ('infrastructure', 'O'), ('areas', 'O'), ('improvements', 'O'), ('around', 'O'), ('hardware', 'I-DEPT'), ('operating', 'O'), ('system', 'O'), ('communications', 'O'), ('software', 'I-DEPT'), ('applications', 'O'), ('data', 'O'), ('processing', 'O'), ('security', 'O'), ('system', 'O'), ('implementing', 'O'), ('develop', 'O'), ('implement', 'O'), ('overall', 'O'), ('security', 'O'), ('ensure', 'O'), ('utmost', 'O'), ('protection', 'O'), ('privacy', 'O'), ('organization

Here we are going to create a list and use set method to get de-duplicated values within the “word” column. And we’ll append the corresponding padding named “Endpad”.

In [7]:
words = list(set(df["words"].values))
words.append("ENDPAD")
n_words  = len(words)

tags = list(set(df["labels"].values))
n_tags = len(tags)

print(n_words , n_tags)

10166 7


### <span style="color:#FF00FF">Converting words to numbers</span>

In [8]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

### <span style="color:#FF00FF">Sequence Padding</span>

In [9]:
max_len = max([len(s) for s in sentences])

X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)
y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [10]:
y = [to_categorical(i, num_classes=n_tags) for i in y]

### <span style="color:#FF00FF">Split train and test data</span>

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### <span style="color:#FF00FF">Build and Compile a Bidirectional LSTM Model</span>

In [12]:
input_word = Input(shape=(max_len,))

model = Sequential()
model = Embedding(input_dim=n_words, output_dim=max_len, input_length=max_len)(input_word)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.0), merge_mode = 'concat')(model)
model = LSTM(units=50, return_sequences=True, dropout=0.5, recurrent_dropout=0.0)(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)
model = Model(input_word, out)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 690)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 690, 690)          7014540   
_________________________________________________________________
dropout (Dropout)            (None, 690, 690)          0         
_________________________________________________________________
bidirectional (Bidirectional (None, 690, 200)          632800    
_________________________________________________________________
lstm_1 (LSTM)                (None, 690, 50)           50200     
_________________________________________________________________
time_distributed (TimeDistri (None, 690, 7)            357       
Total params: 7,697,897
Trainable params: 7,697,897
Non-trainable params: 0
___________________________________________________

In [13]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

### <span style="color:#FF00FF">Train the Model</span>

In [14]:
history = model.fit(x_train, np.array(y_train), batch_size=32, epochs=100, validation_split=0.2, verbose=1)

Epoch 1/100
23/23 [==============================] - 29s 519ms/step - loss: 0.8764 - accuracy: 0.8292 - val_loss: 0.1246 - val_accuracy: 0.9833
Epoch 2/100
23/23 [==============================] - 6s 277ms/step - loss: 0.1256 - accuracy: 0.9823 - val_loss: 0.1079 - val_accuracy: 0.9833
Epoch 3/100
23/23 [==============================] - 6s 279ms/step - loss: 0.1127 - accuracy: 0.9823 - val_loss: 0.0978 - val_accuracy: 0.9833
Epoch 4/100
23/23 [==============================] - 6s 277ms/step - loss: 0.0950 - accuracy: 0.9831 - val_loss: 0.0823 - val_accuracy: 0.9833
Epoch 5/100
23/23 [==============================] - 6s 268ms/step - loss: 0.0862 - accuracy: 0.9823 - val_loss: 0.0712 - val_accuracy: 0.9833
Epoch 6/100
23/23 [==============================] - 7s 287ms/step - loss: 0.0730 - accuracy: 0.9823 - val_loss: 0.0586 - val_accuracy: 0.9834
Epoch 7/100
23/23 [==============================] - 7s 287ms/step - loss: 0.0598 - accuracy: 0.9830 - val_loss: 0.0502 - val_accuracy: 0.984

Epoch 58/100
23/23 [==============================] - 6s 281ms/step - loss: 0.0031 - accuracy: 0.9994 - val_loss: 0.0375 - val_accuracy: 0.9922
Epoch 59/100
23/23 [==============================] - 7s 285ms/step - loss: 0.0032 - accuracy: 0.9994 - val_loss: 0.0363 - val_accuracy: 0.9923
Epoch 60/100
23/23 [==============================] - 7s 284ms/step - loss: 0.0028 - accuracy: 0.9994 - val_loss: 0.0387 - val_accuracy: 0.9922
Epoch 61/100
23/23 [==============================] - 6s 283ms/step - loss: 0.0032 - accuracy: 0.9993 - val_loss: 0.0387 - val_accuracy: 0.9922
Epoch 62/100
23/23 [==============================] - 7s 287ms/step - loss: 0.0030 - accuracy: 0.9994 - val_loss: 0.0357 - val_accuracy: 0.9923
Epoch 63/100
23/23 [==============================] - 7s 291ms/step - loss: 0.0025 - accuracy: 0.9995 - val_loss: 0.0370 - val_accuracy: 0.9923
Epoch 64/100
23/23 [==============================] - 6s 280ms/step - loss: 0.0026 - accuracy: 0.9995 - val_loss: 0.0374 - val_accuracy:

### <span style="color:#FF00FF">Model Evaluation</span>

In [15]:
# Predicting on trained model
pred = model.predict(x_test)
print("Predicted Probabilities on Test Set:\n",pred.shape)
# taking tag class with maximum probability
pred_index = np.argmax(pred, axis=-1)
print("Predicted tag indices: \n",pred_index.shape)

# Flatten both the features and predicted tags for submission
ids,tagids = x_test.flatten().tolist(), pred_index.flatten().tolist()

# converting each word indices back to words
words_test = [words[ind] for ind in ids]
# converting each predicted tag indices back to tags
tags_test = [[tags[ind]] for ind in tagids]
tags_test1 = [tags[ind] for ind in tagids]

Predicted Probabilities on Test Set:
 (229, 690, 7)
Predicted tag indices: 
 (229, 690)


In [16]:
actual_index = np.argmax(y_test, axis=-1)
actual_ids = actual_index.flatten().tolist()
actual_test = [[tags[ind]] for ind in actual_ids]

In [17]:
accuracy = model.evaluate(x_test, np.array(y_test),verbose=0)[1]
f1score  = f1_score(actual_test, tags_test)
precision = precision_score(actual_test, tags_test)
recall = recall_score(actual_test, tags_test)

In [18]:
LSTM_metrics = {
    "F1-score" : f1score,
    "Accuracy" : accuracy,
    "Precision" : precision,
    "Recall" : recall
}
print(LSTM_metrics)

{'F1-score': 0.7067043753119281, 'Accuracy': 0.9891146421432495, 'Precision': 0.7139495798319327, 'Recall': 0.6996047430830039}


### <span style="color:#FF00FF">Classification report</span>

In [19]:
class_report = classification_report(actual_test, tags_test, digits=4)
print(class_report)

              precision    recall  f1-score   support

        DEPT     0.6743    0.6418    0.6577       684
        ROLE     0.6815    0.6852    0.6833       559
       SKILL     0.7389    0.7262    0.7325      1793

   micro avg     0.7139    0.6996    0.7067      3036
   macro avg     0.6983    0.6844    0.6912      3036
weighted avg     0.7138    0.6996    0.7066      3036



In [20]:
pd.Series(tags_test1).value_counts()

O          155035
I-SKILL      1416
I-DEPT        529
I-ROLE        419
B-SKILL       346
B-ROLE        143
B-DEPT        122
dtype: int64

### <span style="color:#FF00FF">Predict on new JD</span>

In [21]:
test_sentence = "information technology"

test_sentence = test_sentence.split()

x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=tag2idx["O"], maxlen=max_len)


p = model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))

Word           ||Prediction
information    : B-DEPT
technology     : I-DEPT


### <span style="color:#FF00FF">Save Model and Supporting Objects</span>

In [22]:
model.save('../models/LSTMmodel.h5')

In [25]:
LSTM_objects = {
    "lstm_word2idx" : word2idx,
    "lstm_tags"     : tags,
    "lstm_max_len"  : max_len,
    "lstm_metrics"  : LSTM_metrics
}
pickle_out = open( "../models/LSTM_objects.pkl", "wb" )
pickle.dump(LSTM_objects, pickle_out)

### <span style="color:#FF00FF">Load Model and Test</span>

In [26]:
pickle_in = open("../models/LSTM_objects.pkl", "rb" )
LSTM_obj = pickle.load(pickle_in)

model_path = "../models/LSTMmodel.h5"
LSTM_model = load_model(model_path)

In [27]:

test_sentence = "java"
test_sentence = test_sentence.split()

x_test_sent = pad_sequences(sequences=[[LSTM_obj.get("lstm_word2idx").get(w, 0) for w in test_sentence]],
                            padding="post", value=tag2idx["O"], maxlen=LSTM_obj.get("lstm_max_len"))

p = LSTM_model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))

Word           ||Prediction
java           : I-SKILL
